In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import torch.cuda
import pandas as pd
from sklearn.metrics import classification_report
from nltk.corpus import stopwords

2024-04-30 19:08:06.231002: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 19:08:06.231047: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 19:08:06.231062: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-30 19:08:06.237624: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 19:08:06.907395: W tensorflow/compiler/

In [2]:
stop_words = stopwords.words('english')
test_df = pd.read_csv('../data/csv/test.csv')

In [3]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [4]:
def encode(examples):
    return tokenizer( examples["text"],
                     truncation=True, 
                     padding=True)

In [5]:
test_df['text'] = test_df['text'].str.lower().str.split().apply(lambda x: [item for item in x if item not in stop_words]).apply(lambda x: " ".join(x)).replace('\d+', '', regex=True)
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(encode, batched=True)

Map:   0%|          | 0/2998 [00:00<?, ? examples/s]

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
id2label = {0: "NON_VIOLATED", 1: "VIOLATED"}
label2id = {"NON_VIOLATED": 0, "VIOLATED": 1}
model = AutoModelForSequenceClassification.from_pretrained("../models/ECHR/roberta_echr_model/removed_stopwords/accuracy/checkpoint-888", num_labels=2, id2label=id2label, label2id=label2id)
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [9]:
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device)

In [10]:
results = pipe(tokenizer.decode(case, clean_up_tokenization_spaces=True, skip_special_tokens=True) for case in test_dataset['input_ids'])

In [11]:
predictions = list(result['label'] for result in results)

In [12]:
report = classification_report(list('NON_VIOLATED' if outcome==0 else 'VIOLATED' for outcome in test_dataset['labels']), predictions)

In [13]:
print(report)

              precision    recall  f1-score   support

NON_VIOLATED       0.84      0.63      0.72      1024
    VIOLATED       0.83      0.94      0.88      1974

    accuracy                           0.83      2998
   macro avg       0.83      0.79      0.80      2998
weighted avg       0.83      0.83      0.83      2998

